# Gamut Mapping - Ramblings 02

In [1]:
%matplotlib widget

In [2]:
from __future__ import division, unicode_literals

import colour
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.optimize
from functools import reduce

COLOUR_STYLE = colour.plotting.colour_style()
COLOUR_STYLE.update({
    'figure.figsize': (10.24, 10.24),
    'legend.framealpha':
    colour.plotting.COLOUR_STYLE_CONSTANTS.opacity.low
})

plt.style.use(COLOUR_STYLE)

plt.style.use('dark_background')

colour.utilities.describe_environment()

colour.utilities.filter_warnings(*[True] * 4);

*                                                                             *
*   Interpreter :                                                             *
*       python : 3.7.6 (default, Dec 30 2019, 19:38:26)                       *
*                [Clang 11.0.0 (clang-1100.0.33.16)]                          *
*                                                                             *
*   colour-science.org :                                                      *
*       colour : v0.3.15-141-g3bebd7e9                                        *
*                                                                             *
*   Runtime :                                                                 *
*       imageio : 2.8.0                                                       *
*       matplotlib : 3.0.3                                                    *
*       numpy : 1.18.4                                                        *
*       scipy : 1.4.1                   

## Colour Stripe Generation

In [3]:
def colour_stripe(samples=360):
    H = np.linspace(0, 1, samples)

    HSV = colour.utilities.tstack([H, np.ones(samples), np.ones(samples)])
    RGB = colour.HSV_to_RGB(HSV)
 
    return RGB[np.newaxis, ...]

In [4]:
COLOUR_STRIPE = colour_stripe();

colour.plotting.plot_image(np.resize(COLOUR_STRIPE, [360, 360, 3]));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Compression Function

In [5]:
def tanh_compression_function(x, a=0.8, b=1 - 0.8):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, (a + b * np.tanh((x - a) / b)), x)

## Comparison

In [6]:
SAMPLES = np.linspace(0, 1, 360)


def sine_wave(x, a, b, c):
    return np.sin(x * a + b) * c


def medicina_HSV_control(p=(np.pi * 6, np.pi, np.pi / 180), RGB=COLOUR_STRIPE, S_c=0, C_f=tanh_compression_function):    
    H, S, V = colour.utilities.tsplit(colour.RGB_to_HSV(RGB))

    HSV = colour.utilities.tstack(
        [(H + sine_wave(SAMPLES, *p))  % 1, C_f(S, S_c, 1 - S_c), V])

    return colour.HSV_to_RGB(HSV)


def medicina_RGB_saturation(RGB=COLOUR_STRIPE, C_t=1, C_f=tanh_compression_function):
    C_t = 1 - C_t

    L = np.max(RGB, axis=-1)[..., np.newaxis]

    D = np.abs(RGB - L) / L

    D_c = C_f(D, C_t, 1 - C_t)

    RGB_c = L - D_c * L

    return RGB_c

RGB_S = medicina_RGB_saturation()

In [7]:
colour.plotting.plot_image(np.resize(medicina_HSV_control(), [360, 360, 3]));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
colour.plotting.plot_image(np.resize(RGB_S, [360, 360, 3]));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
colour.plotting.plot_image(np.resize(np.abs(medicina_HSV_control([0, 0, 0]) - RGB_S) * 5, [360, 360, 3]));

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Optimisation

After some initial tests, it seemed like the closest function to minimize the error was a triangle or sine wave: https://community.acescentral.com/t/gamut-mapping-in-cylindrical-and-conic-spaces/2870/10?u=thomas_mansencal

$\sin\bigg(x * \cfrac{\pi}{6} + \pi\bigg) * \cfrac{\pi}{\approx180})$

is almost the best fitting function for this case, the rounded 180 just happens to be convenient here but the proper value is a bit lower.

In [10]:
def objective_function(abc):
    return np.sum(np.abs(medicina_HSV_control(abc) - RGB_S))


I = 1
def callback(*args):
    global I
    print(I, args[0][:10])
    I += 1

    
P = scipy.optimize.minimize(objective_function, (np.pi * 6, np.pi, np.pi / 180), callback=callback)

1 [  1.88495534e+01   3.14159238e+00   1.78428337e-02]
2 [  1.87968462e+01   3.14659566e+00   1.78300564e-02]
3 [  1.87753957e+01   3.16257721e+00   1.78425503e-02]
4 [  1.87570641e+01   3.18879125e+00   1.78187652e-02]
5 [  1.87635405e+01   3.18574666e+00   1.78591109e-02]
6 [  1.87599644e+01   3.18744775e+00   1.78556345e-02]
7 [  1.87575026e+01   3.18770622e+00   1.78469763e-02]
8 [  1.87582791e+01   3.18733477e+00   1.78409797e-02]
9 [  1.87601454e+01   3.18654866e+00   1.78406727e-02]
10 [  1.87606339e+01   3.18583575e+00   1.78431776e-02]
11 [  1.87600548e+01   3.18632147e+00   1.78458363e-02]
12 [  1.87601760e+01   3.18629586e+00   1.78470703e-02]
13 [  1.87602769e+01   3.18626075e+00   1.78470910e-02]
14 [  1.87603255e+01   3.18622424e+00   1.78469970e-02]
15 [  1.87603343e+01   3.18619542e+00   1.78465309e-02]
16 [  1.87603340e+01   3.18619489e+00   1.78461796e-02]
17 [  1.87603265e+01   3.18620548e+00   1.78462106e-02]
18 [  1.87603266e+01   3.18620585e+00   1.78462050e-02]
1

In [11]:
P

      fun: 1.0626386489805904
 hess_inv: array([[  2.48114583e-03,  -1.09487416e-03,  -7.41781161e-07],
       [ -1.09487416e-03,   5.11710721e-04,   3.01742917e-07],
       [ -7.41781161e-07,   3.01742917e-07,   4.71380077e-10]])
      jac: array([-0.01250617, -0.00483122, -1.91527011])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 692
      nit: 22
     njev: 136
   status: 2
  success: False
        x: array([  1.87603379e+01,   3.18620078e+00,   1.78461965e-02])

In [12]:
colour.plotting.artist()
plt.plot(SAMPLES, np.sin(np.linspace(0, 1, len(SAMPLES)) * P.x[0] + P.x[1]) * P.x[2])
for i in np.arange(0, 360 + 60, 60) / 360:
    plt.axvline(i, c='r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
print(np.max(np.abs(medicina_HSV_control() - RGB_S)))
print(np.max(np.abs(medicina_HSV_control(P.x) - RGB_S)))


colour.plotting.plot_image(np.resize(np.abs(medicina_HSV_control(P.x) - RGB_S) * 5, [360, 360, 3]));

0.00788311318698
0.00808130330299


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …